In [1]:
def __return_waypoints_ind():
    d = np.array([
    [ 65, -36],
    [ 56, -21],
    [ 66, -46],
    [ 58, -48],
    [ 67, -29],
    [ 61, -16],
    [ 45, -32],
    [ 71, -43],
    # [ 80, -52],
    # [ 68, -58],
    [ 65, -54],
    [ 48, -20],
    [ 64, -21],
    [ 46, -14]])
    return d


def __return_waypoints_sdd():
    d = np.array([[24, 34],
       [16, 35],
       [24, 20],
       [14, 20],
       [14, 11],
       [ 6, 33],
       [ 7, 47],
       [23, 48]])
    return d

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os, sys
from helpers.highlevel_sceneloader import HighLevelSceneLoader
from predictors.dataset_creator import TFDataSet
import tensorflow as tf
from predictors.dl_trainer import DLTrainer 
from predictors.extended_predictor import extended_predictor 
import matplotlib.pyplot as plt
import pickle
from helpers.graph import Graph
from helpers.accuracy_functions import destination_distance_l
import numpy as np
from tqdm import tqdm
from math import ceil

import warnings
warnings.filterwarnings('ignore')


''' set some parameters '''
# Model parameters
LSTM_LAYER_SIZE = 32
DENSE_LAYER_SIZE = 32
NUM_LSTM_LAYERS = 2
NUM_DENSE_LAYERS = 2
VARIABLE_INPUT_LENGTH = False

# Dataset
SEQ_IN_LEN = 3
SEQ_OUT_LEN = 10
NOISE_STD = .1
N_REPEATS = 1

BATCH_SIZE = 5
LENGTH_STRIDE = 2


# Training parameters
MAX_EPOCHS = 100
PATIENCE = 10

# For graph
GRAPH_DIST_THRESH = 4

''' get the data '''
ROOT = os.getcwd()

rel_p_img_b = 'helpers/analysed_vars_storage/img_bounds.xml'
rel_p_dests = 'helpers/analysed_vars_storage/destination_locations.xml'
p_img_bounds = os.path.join(ROOT, rel_p_img_b)
p_dest_locs = os.path.join(ROOT, rel_p_dests)

#TODO: older version of OpenTraj needed: "git checkout d249ba6951dd0f54b532fbe2ca6edc46b0d7093f"
opentraj_root = os.path.join(ROOT, 'OpenTraj')
root_datasets = os.path.join(ROOT, 'data/path_data')
sys.path.append(opentraj_root) # add package to pythonpath

scene_data = HighLevelSceneLoader(p_img_bounds, p_dest_locs)
# scene_data.load_ind(root_datasets, 7, 17)
scene_data.load_sdd(opentraj_root, "little", "video3")


''' create the graph instance '''    
interest_points = __return_waypoints_sdd()
g = Graph.from_matrices(interest_points, scene_data.destination_matrix, GRAPH_DIST_THRESH, .05)

df_signals = scene_data.df_to_lst_realxy_mats()
g.analyse_multiple_full_signals(df_signals, add_to_trams_mat=True)


<Element 'dataset' at 0x7f8cec12d0e0>
<Element 'dataset' at 0x7f8cec12d770>


[(array(['w0', 'd6', 'w7'], dtype='<U2'),
  [(24, 34), (25.12452065, 45.36610442), (23, 48)]),
 ([], []),
 ([], []),
 (array(['d0', 'w5', 'w1', 'w0', 'd1'], dtype='<U2'),
  [(0.80188993, 33.06466289),
   (6, 33),
   (16, 35),
   (24, 34),
   (37.4084038, 33.74739443)]),
 (array(['d3', 'd4', 'w2', 'w0'], dtype='<U2'),
  [(18.44346834, 0.86828017), (23.60483923, 0.91962773), (24, 20), (24, 34)]),
 (array(['d7', 'w3', 'w4', 'd3'], dtype='<U2'),
  [(0.29354899, 22.89682134), (14, 20), (14, 11), (18.44346834, 0.86828017)]),
 (array(['d1', 'w0', 'd6', 'w7'], dtype='<U2'),
  [(37.4084038, 33.74739443), (24, 34), (25.12452065, 45.36610442), (23, 48)]),
 (array(['w0', 'w2'], dtype='<U2'), [(24, 34), (24, 20)]),
 (array(['d4', 'w2', 'w0', 'd1'], dtype='<U2'),
  [(23.60483923, 0.91962773), (24, 20), (24, 34), (37.4084038, 33.74739443)]),
 (array(['d2', 'w7', 'd6', 'w0', 'w2', 'd4'], dtype='<U2'),
  [(25.14758057, 53.29443775),
   (23, 48),
   (25.12452065, 45.36610442),
   (24, 34),
   (24, 20),


In [4]:

''' time to create df datasets '''
extra_features_dict = {
    "all_points": None,
    "all_destinations": None,
    "n_destinations": 5,
    "n_points": 5,
    "n_connected_points_after" : 3
}

# Load data in order to not need to do calculations again
with open("data/pickle/ds_creation_d/sdd_3_10.pickle"", 'rb') as handle: #"data/pickle/ds_creation_d/bs1.pickle"
    my_ds_creation_dict = pickle.load(handle)

my_ds = TFDataSet.init_as_fixed_length(scene_data.traj_dataframe, graph=g, var_in_len=VARIABLE_INPUT_LENGTH, length_stride=LENGTH_STRIDE,
scale_list=["pos_x", "pos_y"], seq_in_length=SEQ_IN_LEN, label_length=SEQ_OUT_LEN,
extra_features_dict=extra_features_dict, noise_std=NOISE_STD, 
n_repeats=N_REPEATS, batch_size=BATCH_SIZE, ds_creation_dict=my_ds_creation_dict) # #, save_folder = "data/pickle/ds_creation_d/sdd_3_10.pickle"


100%|██████████| 10/10 [00:05<00:00,  1.72it/s]


Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


In [7]:
''' time for some model training '''

my_trainer_w_epi = DLTrainer(max_epochs=MAX_EPOCHS, patience=PATIENCE)
my_trainer_w_epi.LSTM_one_shot_predictor_named_i(my_ds, 32, 32, 
NUM_LSTM_LAYERS, NUM_DENSE_LAYERS, extra_features=["all_destinations"], 
var_time_len=VARIABLE_INPUT_LENGTH, size_dict=my_ds.size_dict, epistemic=True, dropout_rate=0.05)

from datetime import datetime 

folder_path = "data/model_weights/checkpoints/%s" % (datetime.now().strftime("%d_%m_%Y__%H_%M"))
save_path = os.path.join(folder_path, "model_3_10_sdd_w_dests.pickle")            
if not os.path.exists(folder_path):
    os.mkdir(folder_path)
my_trainer_w_epi.compile_and_fit2(my_ds.tf_ds_dict["train"], my_ds.tf_ds_dict["val"], save_path=save_path)

3
KerasTensor(type_spec=TensorSpec(shape=(None, 3, 2), dtype=tf.float32, name='in_xy'), name='in_xy', description="created by layer 'in_xy'")
KerasTensor(type_spec=TensorSpec(shape=(None, 32), dtype=tf.float32, name=None), name='flatten_2/Reshape:0', description="created by layer 'flatten_2'")
KerasTensor(type_spec=TensorSpec(shape=(None, 9, 3), dtype=tf.float32, name='all_destinations'), name='all_destinations', description="created by layer 'all_destinations'")
Epoch 1/100
373/373 [==============================] - 12s 17ms/step - loss: 0.2686 - mean_absolute_error: 0.3755 - val_loss: 0.1880 - val_mean_absolute_error: 0.3345

Epoch 00001: val_loss improved from inf to 0.18797, saving model to data/model_weights/checkpoints/12_10_2021__23_07/model_3_10_sdd_w_dests.pickle
Alternative save happened on for epoch 0 happened on data/model_weights/checkpoints/12_10_2021__23_07/model_3_10_sdd_w_dests.pickle for val_loss of 0.1880.
Epoch 2/100
373/373 [==============================] - 5s 12m

In [6]:
# Load the 2 models (with and without destination included as a feature)

In [5]:
''' or some model loading '''

# # first without epistemic
# model_path = "data/model_weights/checkpoints/3_10_without_epi/model.pickle"

# my_trainer_wo_epi = DLTrainer(max_epochs=MAX_EPOCHS, patience=PATIENCE)
# my_trainer_wo_epi.LSTM_one_shot_predictor_named_i(my_ds, 32, 32, 
# NUM_LSTM_LAYERS, NUM_DENSE_LAYERS, extra_features=["all_destinations"], 
# var_time_len=VARIABLE_INPUT_LENGTH, size_dict=my_ds.size_dict, epistemic=False, dropout_rate=0.05)
# my_trainer_wo_epi.load_weights(model_path)

# and also one with epistemic
model_path = "data/model_weights/checkpoints/12_10_2021__23_07/model_3_10_sdd_w_dests.pickle"

my_trainer_w_epi = DLTrainer(max_epochs=MAX_EPOCHS, patience=PATIENCE)
my_trainer_w_epi.LSTM_one_shot_predictor_named_i(my_ds, 32, 32, 
NUM_LSTM_LAYERS, NUM_DENSE_LAYERS, extra_features=["all_destinations"], 
var_time_len=VARIABLE_INPUT_LENGTH, size_dict=my_ds.size_dict, epistemic=True, dropout_rate=0.05)
my_trainer_w_epi.load_weights(model_path)

3
KerasTensor(type_spec=TensorSpec(shape=(None, 3, 2), dtype=tf.float32, name='in_xy'), name='in_xy', description="created by layer 'in_xy'")
KerasTensor(type_spec=TensorSpec(shape=(None, 32), dtype=tf.float32, name=None), name='flatten/Reshape:0', description="created by layer 'flatten'")
KerasTensor(type_spec=TensorSpec(shape=(None, 9, 3), dtype=tf.float32, name='all_destinations'), name='all_destinations', description="created by layer 'all_destinations'")


In [49]:
total_probs/num_probs_incl


array([0.33171549, 0.28236701, 0.22567843, 0.15483563, 0.12406456,
       0.09566049, 0.08590556, 0.06971907,        nan,        nan])

In [40]:
num_probs_incl

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [6]:
''' Calculate KPIs for the whole test data set '''
# set up values where 
num_predicted_batches = 0.
total_w_dest = [0.] * len(g.destination_names)
total_wo_dest = [0.] * len(g.destination_names)

test_ds = my_ds.tf_ds_dict["test"]

total_probs = np.zeros(10)
num_probs_incl = np.zeros(10)

for batch in tqdm(iter(test_ds)):
    num_predicted_batches += 1
    batch_in = batch[0]
    batch_out = my_ds.normer.scale_tensor(batch[1], "denormalize", "out") 

    # do prediction with epistemic (batch, num of epi pred, timesteps, x/y)???
    pred_w_epi = my_trainer_w_epi.predict_repetitively_dict_epi_adapted(batch_in, False, 10, False, 35)
    # print(pred_w_epi)
    # print(pred_w_epi.shape)
    # # expanded with 1 destination in first axis
    # pred_w_epi_exp = tf.expand_dims(pred_w_epi, axis=0) 
    print(pred_w_epi.shape)
    # for each element in the batch
    bs = pred_w_epi.shape[0]
    batch_prob_list = []
    for i in range(bs):

        this_epi_prediction = pred_w_epi[i:(i+1),:,:,:]

        print(this_epi_prediction.shape)
        this_label = batch_out[i, :, :]
        for point, point_index in zip(this_label, range(len(this_label))):
            # get results into grid
            grid_resolution = 3
            grid_out, _, grid_limits = scene_data.get_aggregated_output_probs(this_epi_prediction[:,:,point_index:(point_index+1),:], {0:1}, 
            grid_resolution)

            # unpack grid_limits
            x_min = grid_limits[0]
            x_max = grid_limits[1]
            y_min = grid_limits[2]
            y_max = grid_limits[3]

            # Create grid matrix        
            width = ceil((x_max-x_min)/grid_resolution)
            height = ceil((y_max-y_min)/grid_resolution)

            # get probabilities by comparing label to the grid
            # basically: get grid coordinate for each prediction and add the value of it to a list
            
            try:
                grid_x, grid_y = scene_data.find_grid_indices(point[0], point[1], x_min, y_min, grid_resolution)
                
                total_probs[point_index] += grid_out[grid_y, grid_x]
                num_probs_incl[point_index] += 1

            except:
                print("Point %s of the label was out of bound with coordinates (%.1f, %.1f)." % (point_index, point[0], point[1]))


    #     print(total_probs)
    print("Batch: %s" % (total_probs/num_probs_incl))
print("Full set: %s" % (total_probs/num_probs_incl))
    


 

print("With destination: %s"%[el/num_predicted_batches for el in total_w_dest])
print("Without destination: %s"%[el/num_predicted_batches for el in total_wo_dest])



0it [00:00, ?it/s]

(5, 35, 10, 2)
(1, 35, 10, 2)
(1, 35, 10, 2)
(1, 35, 10, 2)
(1, 35, 10, 2)
(1, 35, 10, 2)


1it [00:03,  3.27s/it]

Batch: [0.50285716 0.54285716 0.45714287 0.36571429 0.34285715 0.15428572
 0.29142857 0.00571429 0.11428571 0.        ]
(5, 35, 10, 2)
(1, 35, 10, 2)
(1, 35, 10, 2)
(1, 35, 10, 2)
(1, 35, 10, 2)
(1, 35, 10, 2)


2it [00:06,  3.47s/it]

...

(1, 35, 10, 2)
Batch: [0.33792923 0.3215864  0.28101118 0.18948358 0.13725613 0.10711397
        nan        nan        nan        nan]
(5, 35, 10, 2)
(1, 35, 10, 2)
(1, 35, 10, 2)
(1, 35, 10, 2)
(1, 35, 10, 2)
(1, 35, 10, 2)


110it [06:10,  3.03s/it]

Batch: [0.33901299 0.31980574 0.28043056 0.187761   0.13683952 0.10614021
        nan        nan        nan        nan]
(4, 35, 10, 2)
(1, 35, 10, 2)
(1, 35, 10, 2)
(1, 35, 10, 2)
(1, 35, 10, 2)


111it [06:13,  3.36s/it]

Batch: [0.33929861 0.31863128 0.2795404  0.18753993 0.13616094 0.10573486
        nan        nan        nan        nan]
Full set: [0.33929861 0.31863128 0.2795404  0.18753993 0.13616094 0.10573486
        nan        nan        nan        nan]
With destination: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Without destination: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [30]:
''' time for some model predictions '''
my_predictor_w_dest = extended_predictor(g, my_trainer_w_dest, 1)

nxt_unsc, nxt_sc = my_ds.example_dict("test", "in_xy")

# Let's extract just one path to make visualisation clearer
unscaled_ex = dict(nxt_unsc[0]), nxt_unsc[1]
scaled_ex = dict(nxt_sc[0]), nxt_sc[1]
# let's get an example of length one
for key in unscaled_ex[0].keys():
    unscaled_ex[0][key] = tf.expand_dims(nxt_unsc[0][key][0], axis=0)
for key in scaled_ex[0].keys():
    scaled_ex[0][key] = tf.expand_dims(nxt_sc[0][key][0], axis=0)   

In [31]:
# Basic prediction, but repeated (one at a time)
#PROBLEM: input is scaled
assembled_output, destination_list, dest_prob_dict = my_predictor_w_dest.predict_to_destinations(input_dict=unscaled_ex[0], 
num_steps = 20, variable_len_input=VARIABLE_INPUT_LENGTH, num_predictions=15, norm_probs=True, abs_probs=False)
# Epistemic uncertainty prediction
